In [1]:
import run_model
import create_input_data

import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
input_data = pd.read_csv('xgboost_input.csv')
output_data, model = run_model.run_model()

results = run_model.results(input_data, output_data)
results

/home/fredrik/projects/hack-the-crisis/notebooks/fredrik/run_model.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_change'] = pred
/home/fredrik/projects/hack-the-crisis/notebooks/fredrik/run_model.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_nbr_in_3_days'] = (test['iva']*test['predicted_change']).astype('int')
/home/fredrik/projects/hack-the-crisis/notebooks/fredrik/run_model.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,date,Region,predicted,iva,absolute_error_%
0,2020-03-29,Region Blekinge,0,1.0,1.000000
1,2020-03-30,Region Blekinge,0,1.0,1.000000
2,2020-03-31,Region Blekinge,1,1.0,0.000000
3,2020-04-01,Region Blekinge,1,1.0,0.000000
4,2020-04-02,Region Blekinge,1,2.0,0.500000
...,...,...,...,...,...
95,2020-03-29,Västra Götalandsregionen,45,60.0,0.250000
96,2020-03-30,Västra Götalandsregionen,51,65.0,0.215385
97,2020-03-31,Västra Götalandsregionen,59,70.0,0.157143
98,2020-04-01,Västra Götalandsregionen,67,77.0,0.129870


In [41]:
import plotly.graph_objects as go
import plotly.offline as po

regioner = list(results['Region'].unique())

line = go.Scatter(
    x=results[results['Region'] == regioner[0]]['date'],
    y=results[results['Region'] == regioner[0]]['predicted'],
    name='Predicted IVA'
)

line2 = go.Scatter(
    x=results[results['Region'] == regioner[0]]['date'],
    y=results[results['Region'] == regioner[0]]['iva'],
    name='Actual IVA'
)


updatemenus = [
    {
        'buttons': [
            {
                'method': 'restyle',
                'label': region,
                'args': [
                    {'x': [results[results['Region'] == region]['date']], 
                     'y': [results[results['Region'] == region]['predicted'], 
                           results[results['Region'] == region]['iva']
                          ]
                    },
                ]
            } for region in regioner
            
            
        ],
        'direction': 'down',
        'showactive': True,
    }
]

layout = go.Layout(
    updatemenus=updatemenus,
)

figure = go.Figure(data=[line, line2],  layout=layout)
figure.update_layout(
    xaxis_tickformat='%d %B',
)
figure.write_html('../../plotly_graph.html')
